In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install pydicom
#conda install plotly
conda install scikit-image

%matplotlib inline

import numpy as np
import pandas as pd
from os import listdir, mkdir
import matplotlib.pyplot as plt
from glob import glob #Use Unix shell rules to fine filenames matching a pattern
#from mpl_toolkits.mplot3d.art3d import Poly3DCollection #dealing with 3d polygons
#import scipy.ndimage
#from skimage import morphology #to deal with shape of features in an image
#from skimage import measure
#from skimage.transform import resize
#from sklearn.cluster import KMeans
#from plotly import __version__
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
#from plotly.tools import FigureFactory as FF
#from plotly.graph_objs import *
#init_notebook_mode(connected=True) 

from zipfile import ZipFile
from keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.python.keras.layers import Dense,Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout
from keras.models import Model
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import keras
import tensorflow as tf
from PIL import ImageFile

In [ ]:
listdir("../input/")

In [ ]:
basepath = "../input/rsna-str-pulmonary-embolism-detection/"

train = pd.read_csv(basepath + "train.csv")
test = pd.read_csv(basepath + "test.csv")
train.head()

In [ ]:
#Setting dcm file path as new column in our dataframe:
#train["dcm_path"] = basepath + "train/" + train.StudyInstanceUID + "/" + train.SeriesInstanceUID+ "/"
#test["dcm_path"] = basepath + "test/" + test.StudyInstanceUID + "/" + test.SeriesInstanceUID+ "/"

In [ ]:
#train.head()

In [ ]:
import pydicom
def load_scans(dcm_path):
    # otherwise we sort by ImagePositionPatient (z-coordinate) or by SliceLocation
    slices = [pydicom.dcmread(dcm_path + "/" + file) for file in listdir(dcm_path)]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    return slices

#lets see details for first patient:
example = train.dcm_path.values[0]
scans = load_scans(example)
scans

In [ ]:
import seaborn as sns
fig, ax = plt.subplots(1,2,figsize=(20,5))
for n in range(30):
    image = scans[n].pixel_array.flatten()
    rescaled_image = image * scans[n].RescaleSlope + scans[n].RescaleIntercept
    sns.distplot(image.flatten(), ax=ax[0]);
    sns.distplot(rescaled_image.flatten(), ax=ax[1])
ax[0].set_title("Raw pixel array distributions for 10 examples")
ax[1].set_title("HU unit distributions for 10 examples");
#For some examples we can see that there are raw values at -2000. They correspond to images with a circular boundary within the image. 
#The "outside" of this circle value is often set to -2000

def set_outside_scanner_to_air(raw_pixelarrays):
    # in OSIC we find outside-scanner-regions with raw-values of -2000. 
    # Let's threshold between air (0) and this default (-2000) using -1000
    raw_pixelarrays[raw_pixelarrays <= -1000] = 0
    return raw_pixelarrays

#transformation to HU:

def transform_to_hu(slices):
    images = np.stack([file.pixel_array for file in slices])
    images = images.astype(np.int16)

    images = set_outside_scanner_to_air(images)
    
    # convert to HU
    for n in range(len(slices)):
        
        intercept = slices[n].RescaleIntercept
        slope = slices[n].RescaleSlope
        
        if slope != 1:
            images[n] = slope * images[n].astype(np.float64)
            images[n] = images[n].astype(np.int16)
            
        images[n] += np.int16(intercept)
    
    return np.array(images, dtype=np.int16)

hu_scans = transform_to_hu(scans)
hu_scans[0]

In [ ]:
def get_pixels_hu(scans):
    image = np.stack([s.pixel_array for s in scans])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 1
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    intercept = scans[0].RescaleIntercept
    slope = scans[0].RescaleSlope
    
    if slope != 1:
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)
        
    image += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

In [ ]:
import glob
train_image_file_paths = glob.glob(basepath+'train/' + '/*/*/*.dcm')
test_image_file_paths = glob.glob(basepath +'test/'+ '/*/*/*.dcm')

In [ ]:
#type(train_image_file_paths)

In [ ]:
# Function to take care of teh translation and windowing. 
def window_image(img, window_center,window_width, intercept, slope, rescale=True):
    img = (img*slope +intercept) #for translation adjustments given in the dicom file. 
    img_min = window_center - window_width//2 #minimum HU level
    img_max = window_center + window_width//2 #maximum HU level
    img[img<img_min] = img_min #set img_min for all HU levels less than minimum HU level
    img[img>img_max] = img_max #set img_max for all HU levels higher than maximum HU level
    if rescale: 
        img = (img - img_min) / (img_max - img_min)*255.0 
    return img

def get_first_of_dicom_field_as_int(x):
    #get x[0] as in int is x is a 'pydicom.multival.MultiValue', otherwise get int(x)
    if type(x) == pydicom.multival.MultiValue: return int(x[0])
    else: return int(x)
    
def get_windowing(data):
    dicom_fields = [data[('0028','1050')].value, #window center
                    data[('0028','1051')].value, #window width
                    data[('0028','1052')].value, #intercept
                    data[('0028','1053')].value] #slope
    return [get_first_of_dicom_field_as_int(x) for x in dicom_fields]

def view_images(files, title = '', aug = None, windowing = True):
    width = 2
    height = 2
    fig, axs = plt.subplots(height, width, figsize=(15,15))
    
    for im in range(0, height * width):
        data = pydicom.dcmread(files[im])
        image = data.pixel_array
        window_center , window_width, intercept, slope = get_windowing(data)
        if windowing:
            output = window_image(image, window_center, window_width, intercept, slope, rescale = False)
        else:
            output = image
        i = im // width
        j = im % width
        axs[i,j].imshow(output, cmap=plt.cm.gray) 
        axs[i,j].axis('off')
        
    plt.suptitle(title)
    plt.show()

In [ ]:
view_images(train_image_file_paths[3205:], 'Images with Windowing')

In [ ]:
view_images(train_image_file_paths)

In [ ]:
train_image_file_paths[0]

In [ ]:
listdir(directory)

In [ ]:
#remove folder
#from pathlib import Path
#import shutil

#dirpath = Path('train_imgs')
#if dirpath.exists() and dirpath.is_dir():
#    shutil.rmtree(dirpath)

In [ ]:
#creating a copy of images in new subfolder
import glob 
import shutil

    
destination_path = "test_imgs_folder"
os.mkdir(destination_path)
pattern = "../input/rsna-str-pulmonary-embolism-detection/test/*/*/*"  
for img in glob.glob(pattern):
    shutil.copy(img, destination_path)

In [ ]:
destination_path = "train_imgs_folder"
os.mkdir(destination_path)
pattern = "../input/rsna-str-pulmonary-embolism-detection/train/*/*/*"  
for img in glob.glob(pattern):
    shutil.copy(img, destination_path)

In [ ]:
#move all the train images to one folder
#import os
#import cv2

#directory = '../input/rsna-str-pulmonary-embolism-detection/train'
#new_directory = 'train_imgs_folder'

# If dir does not exist otherwise delete next line
#os.mkdir(new_directory)

#def copy_images():
#    for file_name in train_image_file_paths:
#        img = cv2.imread(file_name[-17:])
#        copied_image_path = new_directory + '/' + file_name
#        cv2.imwrite(copied_image_path, img)

#copy_images()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
#ImageDataGenerator helps in Image Augmentation. IMAGE_SIZE will be used in VGG, ResNet implementation to resize image
#s=32
IMAGE_SIZE = [224, 224]
#set validation split
trainGen=ImageDataGenerator(rescale=1./255,shear_range= 0.2,zoom_range= 0.2,horizontal_flip= True,rotation_range= 40,width_shift_range = 0.2,height_shift_range = 0.2)

testGen = ImageDataGenerator(rescale=1./255)

train_imgs = trainGen.flow_from_directory('train_imgs_folder', class_mode='binary', batch_size=32)
test_imgs = testGen.flow_from_directory('test_imgs_folder', class_mode='binary', batch_size=32)

#train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2, horizontal_flip=True, validation_split=0.2) # set validation split

# same directory as training data
#train_generator = train_datagen.flow_from_directory( train_data_dir,target_size=(img_height, img_width),batch_size=batch_size,class_mode='binary',subset='training') # set as training data
# set as validation data
#validation_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_height, img_width),batch_size=batch_size,class_mode='binary',subset='validation') 


In [ ]:
labels = ['qa_motion','qa_contrast','flow_artifact','true_filling_defect_not_pe']
train = train.drop(labels)

In [ ]:

#Building CNN model:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64,64,3), padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation = 'sigmoid'))
model.summary()
#Lets tell the model what cost and optimization method to use and fit the model to our training data set
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Run the cell. It will take some time to execute


#r= model.fit_generator(train_generator,steps_per_epoch = train_generator.samples // batch_size,validation_data = validation_generator, validation_steps = validation_generator.samples // batch_size,epochs = 100)


#r = model.fit_generator(train_set,validation_data=val_set,epochs=5,steps_per_epoch=len(train_set),validation_steps=len(test_set))

model.fit(train_imgs, train, batch_size = 32, epochs = 100)
model.save('model.h1')

In [ ]:
#test_accuracy = model.evaluate_generator(test,steps=None)

#print('Testing Accuracy-CNN: {:.2f}%'.format(test_accuracy[1] * 100))

In [ ]:
#classes = model.predict_classes(images, batch_size=10)